In [1]:
from debugpy import connect

from GRASP import GRASP

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import dataset_config as dataset_config
from tqdm import tqdm
import math
dataConfig = dataset_config.DatasetConfig()

database = dataConfig.database
poi_dict = dataConfig.poi_dict
database_function = dataConfig.database_func
poi_cate_dict = dataConfig.poi_cate_dict

In [4]:
# 取消输出warning
import warnings
warnings.filterwarnings('ignore')

In [5]:
import json
file_name = "final_datas/1-all_llm_data_output_202409011948_glm4air.json"

with open(file_name,'r', encoding='UTF-8') as f:
     cases_dataset = json.load(f)

all_results = cases_dataset
all_results

[{'request_id': 'request-1',
  'content': {'poi_name_list': [['天津酒店', '天津古文化街', '天津酒店'],
    ['天津酒店', '天津五大道', '天津酒店'],
    ['天津酒店', '天津意式风情区', '天津酒店']],
   'poi_cate_list': [['酒店', '景点', '酒店'],
    ['酒店', '景点', '酒店'],
    ['酒店', '景点', '酒店']],
   'poi_rec_time_list': [[1.0, 2.0, 1.0], [1.0, 2.0, 1.0], [1.0, 2.0, 1.0]],
   'poi_arrive_time_list': [['09:00:00', '09:30:00', '11:30:00'],
    ['09:00:00', '09:30:00', '11:30:00'],
    ['09:00:00', '09:30:00', '11:30:00']]}},
 {'request_id': 'request-2',
  'content': {'poi_name_list': [['天津酒店', '天津古文化街', '耳朵眼炸糕店', '天津文庙', '天津酒店'],
    ['天津酒店', '天津博物馆', '狗不理包子铺', '意式风情区', '天津酒店'],
    ['天津酒店', '瓷房子', '锅巴菜馆', '天津之眼', '天津酒店']],
   'poi_cate_list': [['酒店', '景点', '餐厅', '景点', '酒店'],
    ['酒店', '景点', '餐厅', '景点', '酒店'],
    ['酒店', '景点', '餐厅', '景点', '酒店']],
   'poi_rec_time_list': [[1.0, 2.0, 1.0, 2.0, 1.0],
    [1.0, 2.5, 1.0, 2.0, 1.0],
    [1.0, 2.0, 1.0, 2.5, 1.0]],
   'poi_arrive_time_list': [['09:00:00',
     '10:00:00',
     '12:00:00',
     '1

In [6]:
import json

with open('final_datas/5-analyze_json_input_glm4air_tp0_2_add_classify_layer.json', 'r', encoding='utf-8') as file:
    all_data = json.load(file)

all_data

[{'custom_id': 'request-1',
  'poi_case': {'poi_feature': '人文、历史、文化、多样',
   'route_num': 3,
   'poi_name_list': [],
   'catering_feature': '常规',
   'N_c_min': [1, 2, 2],
   'N_c_max': [10, 3, 2],
   'use_min_restaurant_gap': 10800,
   'time_feature': '慢节奏',
   'start_day_time': '09:00:00',
   'plan_max_time': 12,
   'tightness_w': 1.5},
  'user_query': {'request_id': 'request-1',
   'content': '我打算来一场人文历史之旅，为期3天，用轻松慢节奏的方式去感受各地的风土人情，吃点家常菜，每天从上午9点出发，玩个半天就足够了。'},
  'json_input': {'request_id': 'request-1',
   'content': {'N_c_min': [0, 2, 2],
    'N_c_max': [10, 2, 2],
    'use_min_restaurant_gap': 3,
    'poi_id_list': ['天津古文化街', '天津意式风情区', '天津五大道', '天津张学良故居', '天津瓷房子'],
    'route_num': 3,
    'start_end_poi': None,
    'start_day_time': '09:00:00',
    'plan_max_time': 6,
    'tightness_w': 1.5,
    'user_budget': 1000000000.0,
    'not_poi_list': []}},
  'time_cate': {'request_id': 'request-1', 'content': {'time_feature': '慢节奏'}},
  'catering_cate': {'request_id': 'request-1',
   'cont

In [7]:
def makeUserEmbed(poi_id_list):
    poi_prefers = []
    if len(poi_id_list) == 0:
        poi_prefers.extend(([0]))
    else:
        poi_prefers = poi_id_list
    user_embed = np.zeros(poi_emb_dict[poi_prefers[0]].shape)
    for poi in poi_prefers:
        user_embed += poi_emb_dict[poi]
    user_embed = user_embed / (len(poi_prefers))
    return user_embed

try:
    poi_emb = np.load('data/tianjin_poi_embeddings_20240820192032.npy')
except:
    poi_emb = np.load('../data/tianjin_poi_embeddings_20240820192032.npy')
poi_emb_poi_ids, poi_emb_embeddings = poi_emb[:, 0], poi_emb[:, 1:]
poi_emb_dict = dict(zip(poi_emb_poi_ids, poi_emb_embeddings))

def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

def getPOIEmbed(poi_info):
    return poi_emb_dict[poi_info[poi_dict['poi_id']]]

def getPOIscore(user_emb, poi_info):
    sim = cosine_similarity(user_emb, getPOIEmbed(poi_info))
    return sim * poi_info[poi_dict['poi_score']] * math.log2(poi_info[poi_dict['poi_comment_num']] + 1)

database.connect()
total_score = 0
total_raw_score = 0
total_route_num = 0
for index, value in enumerate(all_data):
    poi_list = [poi_id for poi_id in [item for day_route in all_results[index]['content']['poi_name_list'] for item in day_route]]
    poi_id_list = [item[0] for item in database.getIdbyName(poi_list, "0")]
    poi_info_list = database.getManyPOI(poi_id_list)[0]
    user_emb = makeUserEmbed([item[0] for item in database.getIdbyName(all_data[index]['json_input']['content'].get('poi_id_list', []), "0")])
    total_score += sum([getPOIscore(user_emb, iii) for iii in poi_info_list])
    total_raw_score += sum([poi_info[poi_dict['poi_score']] * math.log2(poi_info[poi_dict['poi_comment_num']] + 1) for poi_info in poi_info_list])
    total_route_num += len(all_results[index]['content']['poi_name_list'])
    
print(total_score)
print(total_route_num)
print(total_score/total_route_num)
print(total_raw_score/total_route_num)
    
database.close()

356458.9266866699
5220
68.2871507062586
82.32729386583694
